In [2]:
import os
import pandas as pd
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.chat_models import ChatOpenAI
import re

In [3]:
# Function to load CSV files
def load_csv_files(file_paths: List[str]) -> Dict[str, pd.DataFrame]:
    return {os.path.basename(f): pd.read_csv(f) for f in file_paths}

In [4]:
 #Custom prompt template for CSV agents
class CSVAgentPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [5]:
# Custom output parser for CSV agents
class CSVAgentOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> AgentAction or AgentFinish:
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentFinish(
                return_values={"output": llm_output.strip()},
                log=llm_output,
            )
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)


In [17]:
# Function to create a CSV agent
# Updated function to create a CSV agent
# Updated function to create a CSV agent
def create_csv_agent(csv_file: str, df: pd.DataFrame):
    def python_repl_with_df(code):
        local_vars = {"df": df, "pd": pd}
        try:
            exec(code, globals(), local_vars)
            return local_vars.get('result', "Code executed but 'result' variable not found.")
        except Exception as e:
            return f"Error executing code: {str(e)}"
    # python_repl = PythonREPL()

    tools = [
        Tool(
            name="Python REPL",
            func=python_repl.run,
            description="""Useful for when you need to execute python code to analyze the CSV data.
                        f"The dataframe is stored in the 'df' variable and represents the {csv_file} file. "
                        "Always store your final result in a variable named 'result'."
                        "If you want to see the output of a value, you should print it out with `print(...)`."""
        )
    ]

    prompt = CSVAgentPromptTemplate(
        template="""You are an intelligent agent designed to analyze the {csv_file} CSV file.
Given an input question, create a plan to figure out the answer and then execute that plan.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to be pythonic and use proper pandas syntax when analyzing the dataframe.
Always store your final result in a variable named 'result'.
Make sure to execute the necessary code to calculate the answer, don't just describe the process.

Question: {input}
{agent_scratchpad}""",
        tools=tools,
        input_variables=["input", "intermediate_steps", "csv_file"]
    )

    llm = OpenAI(temperature=0)
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    tool_names = [tool.name for tool in tools]
    agent = LLMSingleActionAgent(
        llm_chain=llm_chain,
        output_parser=CSVAgentOutputParser(),
        stop=["\nObservation:"],
        allowed_tools=tool_names
    )
    return AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)


In [18]:
# Coordinator agent to manage multiple CSV agents
# Updated coordinator agent to manage multiple CSV agents
# Improved coordinator agent to manage multiple CSV agents
def coordinator_agent(query: str, csv_agents: Dict[str, AgentExecutor]) -> str:
    chat_model = ChatOpenAI(temperature=0)
    response = chat_model.predict_messages([HumanMessage(content=f"""Given the following user query and available CSV files, determine which file(s) are needed to answer the query. Consider the file names and their potential relevance to the query.

User Query: {query}

Available CSV files: {', '.join(csv_agents.keys())}

Respond in the following format:
Files Needed: [List of required CSV files]
Reasoning: [Explain why these files were chosen]
Analysis Plan: [Step-by-step plan to answer the query]
""")])

    plan = response.content
    print("Coordinator's Plan:")
    print(plan)

    files_needed = re.search(r"Files Needed: (.*)", plan)
    if files_needed:
        files_needed = files_needed.group(1).split(', ')
        files_needed = [file.strip('[]').strip() for file in files_needed]
    else:
        files_needed = list(csv_agents.keys())  # If no files specified, use all
    
    results = {}

    for file in files_needed:
        if file in csv_agents:
            # Pass both 'input' and 'csv_file' to the agent
            result = csv_agents[file].run(input=query, csv_file=file)
            results[file] = result
        else:
            print(f"Warning: File '{file}' not found in available agents. Skipping.")

    if len(results) == 1:
        return list(results.values())[0]
    else:
        combined_results = "\n".join([f"{file}: {result}" for file, result in results.items()])
        final_response = chat_model.predict_messages([HumanMessage(content=f"""Given the following results from multiple CSV files, provide a comprehensive answer to the original query. If the results include Python code or data, execute the code or analyze the data to provide a concrete answer.

Original Query: {query}

Results:
{combined_results}

Comprehensive Answer:""")])
        return final_response.content

### MAIN

In [ ]:
# Main execution
if __name__ == "__main__":
    # Simulating file upload
    uploaded_files = [
        "./data/Fib.csv",
    "./data/Obstructive Lung Diseases.csv"
    ]

    # Load CSV files
    dfs = load_csv_files(uploaded_files)

    # Create CSV agents
    csv_agents = {file.lower().replace('_', ' ').replace('.csv', ''): create_csv_agent(file, df) 
                  for file, df in dfs.items()}

    while True:
        user_query = input("Ask a question about the CSV files (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break

        response = coordinator_agent(user_query, csv_agents)
        print("\nFinal Answer:")
        print(response)

In [67]:
# Simulating file upload
uploaded_files = [
    "./data/Fib.csv",
"./data/Obstructive Lung Diseases.csv"
]

# Load CSV files
import pandas as pd
import os
from typing import List, Dict

def load_csv_files(file_paths: List[str]) -> Dict[str, pd.DataFrame]:
    file_data = {}
    
    for path in file_paths:
        file_name = os.path.basename(path)
        file_name_without_ext = os.path.splitext(file_name)[0]
        try:
            if path.endswith('.csv'):
                df = pd.read_csv(path)
            elif path.endswith('.xlsx') or path.endswith('.xls'):
                df = pd.read_excel(path)
            elif path.endswith('.json'):
                df = pd.read_json(path)
            else:
                print(f"Unsupported file format: {path}")
                continue
            file_data[file_name_without_ext] = df
        except Exception as e:
            print(f"Error reading {file_name}: {e}")
    
    return file_data


dfs = load_csv_files(uploaded_files)



In [68]:
# list(dfs['Fib'].columns)

In [81]:
def create_csv_agent(csv_file: str, df: pd.DataFrame):
        print(f"File: {csv_file}, Shape: {df.shape}")
        repl_tool = Tool(name="python_repl", 
                        description="""Useful for when you need to execute python code to analyze the CSV data.
                                f"The dataframe is stored in the 'df' variable and represents the {csv_file} file. "
                                "Always store your final result in a variable named 'result'."
                                "If you want to see the output of a value, you should print it out with `print(...)`.""",
                        func=python_repl.run)
        
        tools = [repl_tool]
        tool_names = [tool.name for tool in tools]
        
        col_names=list(df.columns)
        prompt = """You are an intelligent agent designed to analyze the {csv_file} CSV file.
                Given an input question, create a plan to figure out the answer and then execute that plan.
                You have access to the following tools:

                {tools}

                The csv contains the following columns: {col_names}

                Use the following format:

                Question: the input question you must answer
                Thought: you should always think about what to do
                Action: the action to take, should be one of [{tool_names}]
                Action Input: the input to the action
                Observation: the result of the action
                ... (this Thought/Action/Action Input/Observation can repeat N times)
                Thought: I now know the final answer
                Final Answer: the final answer to the original input question

                Begin! Remember to be pythonic and use proper pandas syntax when analyzing the dataframe.
                Always store your final result in a variable named 'result'.
                Make sure to execute the necessary code to calculate the answer, don't just describe the process.

                Question: {input}
                {agent_scratchpad}"""
                # tools=tools,
                # input_variables=["input", "intermediate_steps", "csv_file"]
                #         )

        llm = ChatOpenAI()
        agent = create_tool_calling_agent(llm, tools, prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

        return agent_executor


# Example
filename1=list(dfs.keys())[0]
pdf=list(dfs.values())[0]

agx= create_csv_agent(filename1, pdf)

In [ ]:
# pdf.head()
agx= create_csv_agent(filename1, pdf)

In [102]:
import pandas as pd
from typing import Dict
# from some_module import Tool, python_repl, ChatOpenAI, create_tool_calling_agent, AgentExecutor  # Ensure these are correctly imported

def create_csv_agent(csv_file: str, df: pd.DataFrame):
    print(f"File: {csv_file}, Shape: {df.shape}")
    
    repl_tool = Tool(
        name="python_repl",
        description="""Useful for when you need to execute python code to analyze the CSV data.
                       The dataframe is stored in the 'df' variable and represents the {csv_file} file.
                       Always store your final result in a variable named 'result'.
                       If you want to see the output of a value, you should print it out with `print(...)`.""",
        func=python_repl.run
    )
    
    tools = [repl_tool]
    tool_names = [tool.name for tool in tools]
    col_names = list(df.columns)
    
    prompt = f"""You are an intelligent agent designed to analyze the {csv_file} CSV file.
Given an input question, create a plan to figure out the answer and then execute that plan.
You have access to the following tools:

{tools}

The csv contains the following columns: {col_names}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{', '.join(tool_names)}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to be pythonic and use proper pandas syntax when analyzing the dataframe.
Always store your final result in a variable named 'result'.
Make sure to execute the necessary code to calculate the answer, don't just describe the process.

Question: {input}
{agent_scratchpad}"""

    llm = ChatOpenAI()  # Ensure the correct instantiation of your language model
    agent = create_tool_calling_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    return agent_executor

    # Example
filename1=list(dfs.keys())[0]
pdf=list(dfs.values())[0]

agx= create_csv_agent(filename1, pdf)

File: Fib, Shape: (24, 39)


NameError: name 'agent_scratchpad' is not defined

In [89]:
from langchain import hub
prompt = hub.pull("homanp/superagent")

In [92]:
prompt

ChatPromptTemplate(input_variables=['input', 'output_format', 'tools'], metadata={'lc_hub_owner': 'homanp', 'lc_hub_repo': 'superagent', 'lc_hub_commit_hash': 'eb77d3d300fe6b1c878305be9ecde4bcf25c49bc0280bd3ab8e39ade23e3c9fc'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output_format', 'tools'], template='# Prompt\n\nObjective:\nYour objective is to create a sequential workflow based on the users query.\n\nCreate a plan represented in JSON by only using the tools listed below. The workflow should be a JSON array containing only the sequence index, function name and input. A step in the workflow can receive the output from a previous step as input.\n\nOutput example 1:\n{output_format}\n\nTools: {tools}\n\nOnly answer with the specified JSON format, no other text')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [100]:
prompt.messages[0].template

AttributeError: 'SystemMessagePromptTemplate' object has no attribute 'template'